In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import dicom
import pydicom
import os
import seaborn as sns
from operator import itemgetter
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import gaussian_kde
import warnings
import plotly 
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

warnings.filterwarnings('ignore')
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
#import plotly.plotly as py

In [ ]:
train_df = pd.read_csv('../data/train_2016_v2.csv',index_col=False, parse_dates=['transactiondate'])
#properties_df = pd.read_csv('../data/properties_2016.csv')
mergelogerror_properties=pd.read_csv('../data/train_merged_class_2016.csv')  
mergelogerror_properties_plotclass=pd.read_csv('../data/train_merged_plotclass_2016.csv') 
mergelogerror_properties_plotclass.head()    
mergelogerror_properties_plotclass.describe()    

In [ ]:
duplicatedtrain=train_df[train_df.duplicated(['parcelid'], keep=False)]
duplicatedtrain.head()

In [ ]:
print(duplicatedtrain['parcelid'].value_counts().describe())
print((duplicatedtrain['parcelid'].value_counts()==3).sum())
print((duplicatedtrain['parcelid'].value_counts()==2).sum())
print(duplicatedtrain['parcelid'].value_counts().head())
doubleduplicated=duplicatedtrain[~(duplicatedtrain['parcelid']==11842707)]
doubleduplicated1=pd.DataFrame(doubleduplicated['logerror'])
errorchange=[]
print (doubleduplicated1.head()) 

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
print(train_df.logerror.describe())
x = train_df.logerror
#data = [go.Histogram(x=x)]
#iplot(data, filename='basic histogram of logerror')

In [ ]:
from sklearn import preprocessing
from pandas.tools.plotting import scatter_matrix
#mergelogerror_properties = pd.merge(properties_df,train_df, on='parcelid')
#%matplotlib inline 
locationproperties=mergelogerror_properties[['longitude','latitude','regionidzip','regionidcity','regionidneighborhood','class']].sample(5000)
locationproperties['longitude']=locationproperties.longitude/1e6
locationproperties['latitude']=locationproperties.latitude/1e6
locationproperties1=locationproperties.dropna()
sns.pairplot(locationproperties1,diag_kind="kde",hue="class",markers='+',plot_kws=dict(s=1,edgecolor="g", linewidth=1),diag_kws=dict(shade=True))

In [ ]:
from plotly.graph_objs import Scatter, Marker, Layout, XAxis, YAxis
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly.plotly as py
sample1=mergelogerror_properties.sample(5000)
lowerror1 = sample1[(sample1['class'] =='lowerror')]
higherror1 = sample1[(sample1['class']=='higherror')]

trace_comp0 = go.Scatter(
    y=lowerror1.latitude/1e6,
    x=lowerror1.longitude/1e6,
    mode='markers',
    marker=Marker(size=lowerror1.logerror, sizemode='area', sizeref=0.01,color='navy'),
    name='lowerror',
    text=lowerror1.logerror,
    )

trace_comp1 = go.Scatter(
    y=higherror1.latitude/1e6,
    x=higherror1.longitude/1e6,
    mode='markers',
    marker=Marker(size=higherror1.logerror, sizemode='area', sizeref=0.01,color='red'),
    name='higherror',
    text=higherror1.logerror,
        )

data_comp = [trace_comp0, trace_comp1]
layout_comp = go.Layout(
    title='Location VS logerror',
    hovermode='closest',
    xaxis=dict(
        title='longitude)',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
    ),
    yaxis=dict(
        title='latitude',
        ticklen=5,
        gridwidth=2,
    ),
)
fig_comp = go.Figure(data=data_comp, layout=layout_comp)
iplot(fig_comp, filename='location vs logerror')

In [ ]:
structioncondition=mergelogerror_properties[['parcelid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt','buildingqualitytypeid','heatingorsystemtypeid','logerror','class']]
structioncondition['lowerror']=[0 if i!='lowerror' else 1 for i in structioncondition['class']]

#print(structioncondition['class'])
errorgroup = pd.crosstab([structioncondition['heatingorsystemtypeid'],
                    structioncondition['buildingqualitytypeid']], 
                    structioncondition['class'])
print(errorgroup)
#errorgroup.plot(kind='bar', stacked=True)
error_rate = errorgroup.div(errorgroup.sum(1).astype(float),
                             axis=0) # normalize the value

# print survival_rate
error_rate.plot(kind='barh', 
                   stacked=True)
plt.title('lowerror and high error percentage by heating system type and building quality type')
plt.show()

In [ ]:
structioncondition=mergelogerror_properties[['parcelid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt','buildingqualitytypeid','heatingorsystemtypeid','logerror','class']]
structioncondition['lowerror']=[0 if i!='lowerror' else 1 for i in structioncondition['class']]
structionconditionnumber=structioncondition[['buildingqualitytypeid','bedroomcnt','bathroomcnt','roomcnt','fullbathcnt','calculatedbathnbr','unitcnt']]
structionconditionnumber.describe()

In [ ]:
timeproperties=mergelogerror_properties[['yearbuilt','assessmentyear','logerror','class']]
timeproperties.describe()

In [ ]:
logerrortimemean=timeproperties[['logerror','yearbuilt']].groupby('yearbuilt'). mean()
logerrortimemean.describe()

In [ ]:
timeproperties=timeproperties[['yearbuilt','logerror','class']]
timeproperties.head()

In [ ]:
train_df['month']=train_df['transactiondate'].dt.month
train_df['month'].head()

In [ ]:
trainlogerrormean=train_df[['logerror','month']].groupby('month'). mean()
trainlogerrormean.describe()

In [ ]:
abstrain_2016=train_df
abstrain_2016['logerror']=abs(train_df['logerror'])
abstrain_2016mean=abstrain_2016[['logerror','month']].groupby('month'). mean()
abstrain_2016mean.head()

In [ ]:
#mergelogerror_properties['month']=mergelogerror_properties['transactiondate'].dt.month
mergelogerror_properties['transactiondate'] = pd.to_datetime(mergelogerror_properties['transactiondate'], errors='coerce')
mergelogerror_properties['month']=mergelogerror_properties['transactiondate'].dt.month
#mergelogerror_properties['transactiondate'][0]
mergelogerror_properties.head()

In [ ]:
areapropertyland=mergelogerror_properties[['parcelid','calculatedfinishedsquarefeet','finishedsquarefeet12','lotsizesquarefeet','taxamount','taxvaluedollarcnt','structuretaxvaluedollarcnt','propertycountylandusecode','propertyzoningdesc','landtaxvaluedollarcnt','logerror','class']]
areapropertyland.describe()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition.pca import PCA
from sklearn import preprocessing
df=mergelogerror_properties[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror']].copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet']]

print(df_scaled.head())
y = df.logerror
target_names='logerror'


 # fit data and then transform it
for k in range(1,6):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    print('explained_variance_ratio:',pca.explained_variance_ratio_)
    print('pca:',pca.components_)

In [ ]:
#import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.tools as tls
from sklearn.preprocessing import StandardScaler
X=df_scaled
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
#u,s,v = np.linalg.svd(X_std.T)
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
# Make a list of (eigenvalue, eigenvector) tuples

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)


#fig = Figure(data=data1, layout=layout)
#iplot(Figure(data=data1, layout=layout))

In [ ]:
import plotly.graph_objs as go
trace1 = go.Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
#    ,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

In [ ]:
trace1 = go.Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

In [ ]:
X=df_scaled[['fullbathcnt','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
#u,s,v = np.linalg.svd(X_std.T)


In [ ]:
#import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))


eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)
trace1 = go.Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

fig = Figure(data=data1, layout=layout)
iplot(Figure(data=data1, layout=layout))

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
X=df_scaled[['fullbathcnt','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
X_std = StandardScaler().fit_transform(X)
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
#print('Covariance matrix \n%s' %cov_mat)
#print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat1 = np.corrcoef(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat1)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
cor_mat2 = np.corrcoef(X.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
#print('Eigenvectors \n%s' %eig_vecs)
#print('\nEigenvalues \n%s' %eig_vals)
#u,s,v = np.linalg.svd(X_std.T)
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
# Make a list of (eigenvalue, eigenvector) tuples

eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('the cum')
print(cum_var_exp)
trace1 = go.Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data1 = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Number of rooms variables: Explained variance by different principal components ')

fig = Figure(data=data1, layout=layout)
iplot(Figure(data=data1, layout=layout))

In [ ]:

mergelogerror_properties_plotclass['yearbuilt'] = pd.to_datetime(mergelogerror_properties_plotclass['yearbuilt'], errors='coerce')
type(mergelogerror_properties_plotclass['yearbuilt'][0])

In [ ]:
#meansd=abs(-0.16107794320832886/2+0.011457219606756682)
#train_df['plotclass'] = pd.cut(abs(train_df.logerror),[0,meansd,1e6],3,
#                                 labels=['0','1']) # this creates a new variable
#mergelogerror_properties_plotclass = pd.merge(properties_df,train_df, on='parcelid')
df=mergelogerror_properties_plotclass[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror','plotclass']].sample(1000).copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','taxamount','lotsizesquarefeet']]
#print(df.plotclass.describe())
y=df.plotclass

target_names='logerror'
 # fit data and then transform it
for k in range(1,6):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    #print('explained_variance_ratio:',pca.explained_variance_ratio_)
    #print('pca:',pca.components_)
    from pandas.plotting import scatter_matrix
def get_feature_names_from_weights(weights, names):
    tmp_array = []
    for comp in weights:
        tmp_string = ''
        for fidx,f in enumerate(names):
            if fidx>0 and comp[fidx]>=0:
                tmp_string+='+'
            tmp_string += '%.2f*%s ' % (comp[fidx],f[:-5])
        tmp_array.append(tmp_string)
    return tmp_array
  
pca_weight_strings = get_feature_names_from_weights(pca.components_, df_scaled.columns) 
df_pca = pd.DataFrame(X_pca,columns=[pca_weight_strings])
y=preprocessing.scale(y)
ax = scatter_matrix(df_pca, pca_weight_strings[0], pca_weight_strings[1], c=y,s=(y+2)*10)


In [ ]:
pca_weight_strings[0]

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.decomposition.pca import PCA
from sklearn import preprocessing
df=mergelogerror_properties_plotclass[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt','plotclass']].copy()
df=df.dropna()
df_scaled = preprocessing.scale(df[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['bedroomcnt','bathroomcnt','fullbathcnt','calculatedbathnbr','unitcnt']]

print(df_scaled.head())
y = df.plotclass
target_names='plotclass'

for k in range(1,3):
    pca=PCA(n_components=k)
    X_pca = pca.fit(df_scaled).transform(df_scaled)
    pca.fit(df_scaled)
    print('explained_variance_ratio:',pca.explained_variance_ratio_)
    print('pca:',pca.components_)

from pandas.tools.plotting import scatter_matrix
def get_feature_names_from_weights(weights, names):
    tmp_array = []
    for comp in weights:
        tmp_string = ''
        for fidx,f in enumerate(names):
            if fidx>0 and comp[fidx]>=0:
                tmp_string+='+'
            tmp_string += '%.2f*%s ' % (comp[fidx],f[:-5])
        tmp_array.append(tmp_string)
    return tmp_array
  
pca_weight_strings = get_feature_names_from_weights(pca.components_, df_scaled.columns) 
df_pca = pd.DataFrame(X_pca,columns=[pca_weight_strings])
y=preprocessing.scale(y)
ax = scatter_matrix(df_pca, pca_weight_strings[0], pca_weight_strings[1], c=y,s=(y+2)*10)